<div style="padding: 30px 25px; border: 2px #6495ed solid">
    
<b>Привет, Николай!</b>

Меня зовут Алексей Косенко, и я буду проверять твой проект. Прошу прощения за долгое ожидание
    
Предлагаю общаться на «<b>ты</b>», но если это неудобно — обязательно сообщи об этом.
    
Ниже ты увидишь мои комментарии, выделенные разным цветом:
    
<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

<b>Отлично!</b> ✔️: все ок, продолжай в том же духе :)

</div>
<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
<b>Есть совет</b> ⚠️: так выделены некритичные замечания или советы, которые смогут улучшить твой проект.

</div>
<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

<b>Нужно доработать</b> ❌: так выделены важные замечания — без их исправления проект не может быть принят.
    
</div>

Будет здорово, если ты будешь участвовать в диалоге, а свои вопросы, комментарии и изменения в проекте тоже будешь выделять, например, вот так:
    
<div class="alert alert-info">
    <b>Комментарий студента:</b> твой комментарий
</div>
    
Пожалуйста, не перемещай и не удаляй мои комментарии, это усложнит повторную проверку.
    
</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Привет! Я буду делать вот такие комментарии</font>
</div>

## Проект спринт 10
**Название проекта**:  
Отток клиентов "Бета-Банка"   
_____
**Описание исследования**:  
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.
_____
**Цель исследования**:  
Необходимо построить модель прогноза ухода клиента из банка в ближайшее время.
_____
**План исследования.**  
[Шаг 1. Загрузка и подготовка данных](#step1)  
[Шаг 2. Исследование баланса классов](#step2)  
[Шаг 3. Обучение модели без учёта дисбаланса](#step3)  
[Шаг 4. Улучшение качества модели с учетом дисбаланса классов](#step4)  
[Шаг 5. Финальное тестирование и окончательный выбор модели](#step5)  
[Шаг 6. Результаты и выводы](#step6)  
_____
**Исходные данные.**
Исходный файл <a HREF='Churn.csv'>Churn.csv</a>, содержащий исторические данные о поведении клиентов и расторжении договоров с банком.  
поля файла:
Признаки
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата
Целевой признак
- Exited — факт ухода клиента

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: молодец, что оформляешь описание!</div>

<a id='step1'></a>
## Шаг 1. Загрузка и подготовка данных

In [ ]:
# загрузка библиотек
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
RANDOM_STATE = 12345

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: все импорты в одном месте, удобно :)</div>

In [ ]:
try:
    df = pd.read_csv('Churn.csv')
    print('Загрузили файл локально')
except:
    print('локально не удалось, загружаем из интернета')
    df = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

In [ ]:
display(df.info())
display(df.sample(10))

In [ ]:
sns.pairplot(df)

In [ ]:
df.CreditScore.hist(bins=30)

In [ ]:
df.Age.hist(bins=60, figsize=(15, 10))

In [ ]:
df.Balance.hist()

### первые наблюдения касательно качества данных  
1. Имеется 909 пропусков в поле `Tenure`
2. поля `RowNumber` и `CustomerId` не должны влиять на уход клиента из банка
3. поле `Surname` может являться важной информацией: возможна связь между членами семьи. Однако поскольку внедрение модели предполагает оценку одного конкретного клиента, поле `Surname` также будем считать неважным для модели прогноза
4. Заметны несколько аномалий
   1. Большое количество людей с максимальным кредитным рейтингом - это вполне объяснимо
   2. Отклонения от нормальной кривой распределения возраста - видны максимумы в возрастах 23, 29, 33, 40, 44, 50,55, 60, 70 лет
   3. Большое количество клиентов с нулевым остатком на счете
### план подготовки данных
1. исследовать `Tenure` и заполнить пропуски
2. удалить колонки `RowNumber`, `CustomerId` и `Surname`
3. разделить датасет на фичи и таргеты и разделить на обучающую, валидационную и тестовую выборки
4. сконвертировать поля `Geography` и `Gender` в категориальные переменные методом OHE
5. Нормализовать численные поля `CreditScore`, `Age`, `Tenure`, `Balance`, `NumOfProducts`, `EstimatedSalary`

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: молодец, что подробно изучаешь данные!</div>

In [ ]:
df[df.Tenure.isna()].sample(20)

In [ ]:
df.Tenure.describe()

Особенностей незаметно, заполним пропущенные значения медианой

In [ ]:
m = df.Tenure.median()
df.loc[df.Tenure.isna(),'Tenure'] = m

In [ ]:
df.Tenure.describe()

немного уменишилось стандратное отклонение, в остальном без изменений

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: заполнить пропуски медианой можно, но стоит проверить, что такое заполнение не исказит распределение признака. Для этого можно, например, проверить характеристики распределения(mean, std, median) до заполнения и после, и проверить, что они изменились незначительно</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>добавил проверку чуть выше</font>
</div>

In [ ]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: согласен, эти признаки для моделей не информативны</div>

In [ ]:
features = df.drop('Exited', axis='columns')
target = df['Exited']

In [ ]:
features_, features_test, target_, target_test =\
    train_test_split(features, target, test_size=0.2, stratify=target, random_state=RANDOM_STATE)
features_train, features_valid, target_train, target_valid =\
    train_test_split(features_, target_, test_size=0.25, stratify=target_, random_state=RANDOM_STATE)
print(f'features_train: {features_train.shape},\ttarget_train: {target_train.shape}')
print(f'features_valid: {features_valid.shape},\ttarget_valid: {target_valid.shape}')
print(f'features_test: {features_test.shape},\ttarget_test: {target_test.shape}')


In [ ]:
display(df.Geography.unique())
display(df.Gender.unique())

In [ ]:
categorial = ['Geography', 'Gender']
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
ohe = OneHotEncoder(drop='first', sparse=False)
ohe.fit(features_train[categorial])

In [ ]:
ohe.categories_

In [ ]:
# процедура для One-Hot кодирования отдельной выборки.
# OneHotEncoder работает проще, чем pandas.get_dummies, поэтому надо
# ohe.transform возвращает врое массив, но точно не DataFrame
# поэтому делает из него DataFrame, добавляя названия полей из Ohe и (важно) index из первоначального
# датафрейма, иначе при склейке исходного датафрейма с ohe значения ohe не сойдутся

def ohe_transfrom(data, ohe):
    global categorial
    data_ohe = ohe.transform(data[categorial])
    data_ohe = pd.DataFrame(data_ohe, columns=ohe.get_feature_names(), index=data.index)
    # display(data_ohe)
    data[ohe.get_feature_names()] = data_ohe 
    data.drop(columns=categorial, inplace=True)
    # display(data)
    return data

In [ ]:
features_train = ohe_transfrom(features_train, ohe)
features_valid = ohe_transfrom(features_valid, ohe)
features_test = ohe_transfrom(features_test, ohe)
features_train

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: окей, OHE в этом случае подходит, но правильнее обучать кодировщик только на обучающей выборке, а трансформировать уже все, аналогично масштабированию, чтобы избежать утечки данных. Тогда вместо get_dummies можно использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html" target="blank_">OneHotEncoder</a>, подробнее про различия можно почитать <a href="https://albertum.medium.com/preprocessing-onehotencoder-vs-pandas-get-dummies-3de1f3d77dcc" target="blank_">здесь</a>

Обрати внимание, что в следующих проектах это будет считаться критичной ошибкой</div>

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: здесь нужно разбить данные на три выборки: обучающую, валидационную и тестовую. Обучающая выборка нужна для обучения модели, валидационная — для подбора гиперпараметров, сравнения метрик и выбора лучшей модели, а тестовая для проверки работоспособности лучшей модели на этапе тестирования. Это можно сделать с помощью двух последовательных вызово train_test_split. Обрати внимание, что выборки не должны пересекаться. Как правило, отношение размеров таких выборок — 60/20/20</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>учтено: код разделения изменен на 3 выборки, помещен ДО секций с трансформациями данных</font>
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: так как в целевом признаке есть дисбаланс, выборки лучше стратифицировать с помощью параметра **stratify** в train_test_split. Так у тебя точно не возникнет ситуации, когда, например, в тестовой выборке соотношение классов будет сильно отличаться от соотношения в обучающей выборке

Подробнее про стратификацию:
* https://stackoverflow.com/questions/29438265/stratified-train-test-split-in-scikit-learn
* https://code.likeagirl.io/good-train-test-split-an-approach-to-better-accuracy-91427584b614    
* https://habr.com/ru/post/496750/
* https://habr.com/ru/company/X5Group/blog/596279/
</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>учтено: параметр stratify установлен в процедурах train_test_split</font>
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера <a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: значение random_state лучше вынести в константу, например, RANDOM_STATE. Это общее правило код-стайла не только в Практикуме, а в большинстве языков программирования: мы выносим все константные значения в переменные/константы с читаемым названием и именуем, согласно стандартам PEP-8, заглавными буквами. Так проще читать и дебажить код. Это не обязательное условие, просто полезная привычка на будущее</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>добавил присвоение в секцию загрузки библиотек и заменил везде 12345 на RANDOM_STATE</font>
</div>

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train.loc[:,numeric] = scaler.transform(features_train[numeric])
features_valid.loc[:,numeric] = scaler.transform(features_valid[numeric])
features_test.loc[:,numeric] = scaler.transform(features_test[numeric])

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: молодец, что не забываешь про масштабирование!</div>

In [ ]:
features_train.sample(10)

<a id='step2'></a>
## Шаг 2. Исследование баланса классов

In [ ]:
display(df.Exited.hist())
display(df.Exited.value_counts())

Целевые классы несбалансированны: из выборки в 10 тысяч клиентов почти 8 тысяч остаются, чуть больше 2 тысяч уходят, соотношение примерно 4:1

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: все так, жуткий дисбаланс</div>

<a id='step3'></a>
## Шаг 3. Обучение модели без учёта дисбаланса

Будем обучать и искать лучшие параметры для 2 типов моделей: RandomForestClassifier, SVC

In [ ]:
rfc_params = {
    'n_estimators': range (10, 51, 10),
    'max_depth': range (1, 15)
}
rfc = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), rfc_params, scoring='f1')
rfc.fit(features_train, target_train)

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: круто, что используешь GridSearch!</div>

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: но GridSearchCV для выбора лучшей модели использует в качестве метрики ту, которая используется для метода score в самой модели. В случае выбранных тобой моделей — это accuracy_score. В нашей задаче для подбора гиперпараметров нужна F1-мера. Для выбора метрики в GridSearchCV используется параметр scoring, подробнее можно узнать в <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html" target="_blank">документации</a></div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>учтено, исправлено, теперь все значения метрик другие, для простоты отказался от логистической регрессии</font>
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: также передавать списки с одним значением гиперпараметра в GridSearchCV не стоит, ведь перебирать их не нужно, лучше указать их при инициализации модели</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>учтено</font>
</div>

In [ ]:
display(rfc.best_params_)
display(rfc.best_score_)
rfc = rfc.best_estimator_

In [ ]:
svc_params = {
    'C': [1, 10],
    'gamma': [1, 0.1],
    'kernel': ['linear', 'rbf']
}
svc = GridSearchCV(SVC(random_state=RANDOM_STATE, probability=True), svc_params, scoring='f1')
svc.fit(features_train, target_train)

In [ ]:
display(svc.best_params_)
display(svc.best_score_)
svc = svc.best_estimator_

рассчитаем метрики F1 и ROC-AUC для моделей с лучшими параметрами на валидационной выборке

In [ ]:
pred_valid = rfc.predict(features_valid)
pred_valid_a = rfc.predict_proba(features_valid)
print(f'F1 = {f1_score(target_valid, pred_valid)}, ROC-AUC={roc_auc_score(target_valid, pred_valid_a[:,1])}')

In [ ]:
pred_valid = svc.predict(features_valid)
pred_valid_a = svc.predict_proba(features_valid)
print(f'F1 = {f1_score(target_valid, pred_valid)}, ROC-AUC={roc_auc_score(target_valid, pred_valid_a[:,1])}')

на обучающей выборке получаются следующие результаты:
|модель|метрика на обучающей выборке|лучшие параметры|Метрики на валидационной выборке|
|-|-|-|-|
|RandomForestClassifier|F1 = 0.57|'max_depth': 13, 'n_estimators': 20|F1 = 0.57, ROC-AUC = 0.85
|SVC|F1 = 0.55|'C': 10, 'gamma': 0.1, 'kernel': 'rbf'|F1 = 0.54, ROC-AUC = 0.82

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: для расчета ROC-AUC необходимо использовать вероятности класса 1, полученные методом `predict_proba`, как было в тренажере, нужно поправить

Также обрати внимание, что дополнительно к F1-мере в этом проекте нужно измерять AUC-ROC на настроенных моделях и сравнивать изменения F1 и AUC-ROC в выводах на **каждом** этапе. Так мы сможем проверить, как меняются разные метрики в зависимости от выбранной модели и способа борьбы с дисбалансом
</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>доработано</font>
</div>

<a id='step4'></a>
## Шаг 4. Улучшение качества модели с учетом дисбаланса классов

In [ ]:
def print_model_params_valid(model):
    global features_valid
    global target_valid
    pred_valid = model.predict(features_valid)
    pred_valid_a = model.predict_proba(features_valid)
    f1 = f1_score(target_valid, pred_valid)
    roc_auc = roc_auc_score(target_valid, pred_valid_a[:, 1])
    print(f'F1 = {f1}, ROC-AUC = {roc_auc}, params = {model.get_params(deep=False)}')

сбалансируем целевые классы путем upsampling класса 1

In [ ]:
UPSAMPLING_COEF = 4
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

features_upsampled = pd.concat([features_zeros] + [features_ones] * UPSAMPLING_COEF)
target_upsampled = pd.concat([target_zeros] + [target_ones] * UPSAMPLING_COEF)

features_upsampled_train, target_upsampled_train = shuffle(features_upsampled, target_upsampled, random_state=RANDOM_STATE)

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: коэффициент увеличения выборки подобран верно!</div>

In [ ]:
def GridSearchWoCV(model, params, features_train, target_train, features_valid, target_valid):
    if len(params)==0:
        model.fit(features_train, target_train)
        pred_valid = model.predict(features_valid)
        pred_valid_a = model.predict_proba(features_valid)
        f1 = f1_score(target_valid, pred_valid)
        roc_auc = roc_auc_score(target_valid, pred_valid_a[:, 1])
        # TO-DO return
        return model, f1, roc_auc
    else:
        best_model = None
        best_f1 = -1
        best_roc_auc = -1
        params_copy = params.copy()
        this_param = params_copy.popitem()
        for param in this_param[1]:
            model_param = {this_param[0]: param}
            model.set_params(**model_param)
#            print(param)
            m, f1, roc_auc = GridSearchWoCV(model, params_copy, features_train, target_train, features_valid, target_valid)
#            print(f1)
            if f1 > best_f1:
                best_f1 = f1
                best_model = m
                best_roc_auc = roc_auc
        return best_model, best_f1, best_roc_auc


In [ ]:
rfc_params = {
    'n_estimators': range (10, 51, 10),
    'max_depth': range (1, 15),
}
rfc_upsampled, best_f1, best_roc_auc = GridSearchWoCV(RandomForestClassifier(random_state=RANDOM_STATE), rfc_params, features_upsampled_train, target_upsampled_train, features_valid, target_valid)
print_model_params_valid(rfc_upsampled)

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: обрати внимание, что GridSearchCV использует кросс-валидацию для расчета метрик, подробнее про кросс-валидацию расскажут в следующем спринте. Но на upsample выборках нельзя проводить кросс-валидацию, потому что модель очень вероятно переобучится(что и произошло), <a href="https://datascience.xyz/practice/kak-delat-kross-validaciju-pri-apsemplinge-dannyh.html" target="blank_">здесь</a> можно почитать про это подробнее. Тут лучшей стратегией будет отказаться от кросс-валидации для этой проверки, либо реализовать собственную кросс-валидацию, либо попробовать другие инструменты, как описано в статье</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Исправлено: сделал самостоятельно GridSearch без кросс-валидации, т.е. на валидационной выборке</font>
</div>

In [ ]:
svc_params = {
    'C': [1, 10],
    'gamma': [1, 0.1],
    'kernel': ['linear', 'rbf']
    }
svc_upsampled, best_f1, best_roc_auc = GridSearchWoCV(SVC(random_state=RANDOM_STATE, probability=True), svc_params, features_upsampled_train, target_upsampled_train, features_valid, target_valid)
print_model_params_valid(svc_upsampled)

на валидационной выборке получаются следующие результаты:
|модель|F1|ROC-AUC|лучшие параметры|
|-|-|-|-|
|RandomForestClassifier|0.63|0.87|'max_depth': 14, 'n_estimators': 50|
|SVC|0.60|0.86|'C': 10, 'gamma': 0.1, 'kernel': 'rbf'|

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: для честного эксперимента нужно проверить еще хотя бы один способ борьбы с дисбалансом на каждой модели(взвешивание классов, downsampling, изменение порога классификации)</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>доработано: ниже обучение моделей с взвешиванием классов</font>
</div>

применим взвешивание классов и проверим качество моделей

In [ ]:
rfc_params = {
    'n_estimators': range (10, 51, 10),
    'max_depth': range (1, 15)
}
rfc = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced'), rfc_params, scoring='f1')
rfc.fit(features_train, target_train)

In [ ]:
display(rfc.best_params_)
display(rfc.best_score_)
rfc_balanced = rfc.best_estimator_

In [ ]:
pred_valid = rfc_balanced.predict(features_valid)
pred_valid_a = rfc_balanced.predict_proba(features_valid)
print(f'F1 = {f1_score(target_valid, pred_valid)}, ROC-AUC={roc_auc_score(target_valid, pred_valid_a[:,1])}')

In [ ]:
svc_params = {
    'C': [1, 10],
    'gamma': [1, 0.1],
    'kernel': ['linear', 'rbf']
}
svc = GridSearchCV(SVC(random_state=RANDOM_STATE, probability=True, class_weight='balanced'), svc_params, scoring='f1')
svc.fit(features_train, target_train)

In [ ]:
display(svc.best_params_)
display(svc.best_score_)
svc_balanced = svc.best_estimator_

In [ ]:
pred_valid = svc_balanced.predict(features_valid)
pred_valid_a = svc_balanced.predict_proba(features_valid)
print(f'F1 = {f1_score(target_valid, pred_valid)}, ROC-AUC={roc_auc_score(target_valid, pred_valid_a[:,1])}')

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: еще для наглядности не хватает таблички с результатами, где будет название модели, примененный способ борьбы с дисбалансом, лучшие гиперпараметры и значения метрик F1 и ROC-AUC</div>
<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Доработано: ниже такая табличка</font>
</div>

Итоговые метрики по моделям на валидационной выборке в зависмости от способа борьбы с дисбалансом целевых классов:  
|Модель|исправление дисбаланса|F1|ROC-AUC|лучшие гипер-параметры|
|-|-|-|-|-|
|RandomForest|balanced|0.63|0.86|'max_depth': 10, 'n_estimators': 50|
|SVC|balanced|0.61|0.86|'C': 1, 'gamma': 0.1, 'kernel': 'rbf'|
|RandomForest|upsampling|0.63|0.87|'max_depth': 14, 'n_estimators': 50|
|SVC|upsampling|0.60|0.86|'C': 10, 'gamma': 0.1, 'kernel': 'rbf'|


<a id='step5'></a>
## Шаг 5. Финальное тестирование и окончательный выбор модели

рассчитаем метрики F1 и ROC-AUC для моделей с лучшими параметрами на валидационной выборке, а именно, RandomForest:  
- обученной с учетом взвешивания целевых классов
- обученной с применением upsampling

In [ ]:
pred_test = rfc_balanced.predict(features_test)
pred_test_a = rfc_balanced.predict_proba(features_test)
print(f'F1 = {f1_score(target_test, pred_test)}, ROC-AUC={roc_auc_score(target_test, pred_test_a[:,1])}')

In [ ]:
pred_test = rfc_upsampled.predict(features_test)
pred_test_a = rfc_upsampled.predict_proba(features_test)
print(f'F1 = {f1_score(target_test, pred_test)}, ROC-AUC={roc_auc_score(target_test, pred_test_a[:,1])}')

выберем модель RandomForestClassifier, обученную на сбалансированных данных  
модель показывает достаточную предсказательную силу:  
метрики F1 = 0.62, ROC-AUC=0.86

<a id='step6'></a>
## Шаг 6. Результаты и выводы

Были проанализированы данные по уходу клиентов из банка.  
В полученной выборке остающихся клиентов примерно в 4 раза больше, чем уходящих.  
Сначала были построены модели для дальнейшей работы без учета дисбаланса классов  
были получены следующие результаты:
|модель|метрика на обучающей выборке|лучшие параметры|Метрики на валидационной выборке|
|-|-|-|-|
|RandomForestClassifier|F1 = 0.57|'max_depth': 13, 'n_estimators': 20|F1 = 0.57, ROC-AUC = 0.85
|SVC|F1 = 0.55|'C': 10, 'gamma': 0.1, 'kernel': 'rbf'|F1 = 0.54, ROC-AUC = 0.82

Далее было проведено обучение моделей на искусственно сбалансированных данных.  
Результаты на валидационной выборке:
|Модель|исправление дисбаланса|F1|ROC-AUC|лучшие гипер-параметры|
|-|-|-|-|-|
|RandomForest|balanced|0.63|0.86|'max_depth': 10, 'n_estimators': 50|
|SVC|balanced|0.61|0.86|'C': 1, 'gamma': 0.1, 'kernel': 'rbf'|
|RandomForest|upsampling|0.63|0.87|'max_depth': 14, 'n_estimators': 50|
|SVC|upsampling|0.60|0.86|'C': 10, 'gamma': 0.1, 'kernel': 'rbf'|

Для выбора модели были рассчитаны метрики F1 и ROC-AUC для двух моделей на тестовых данных.  
Были получены следующие результаты:
|модель|F1|ROC-AUC|
|-|-|-|
|RandomForestClassifier balanced|0.62|0.86|
|RandomForestClassifier upsampled|0.57|0.85|  

В итоге выбрана модель RandomForestClassifier, обученная на сбалансированных данных.  
Модель показывает достаточную предсказательную силу и ее можно использовать для предсказания оттока клиентов из Бета-Банка


<div style="padding: 20px 25px; border: 2px #6495ed solid">
    
<h2 style="color: #87187D">Итоговый комментарий ревьюера<a class="tocSkip"></a></h2>

Видно, что ты приложил много усилий, спасибо за проект!

Но тебе обязательно нужно обратить внимание на следующее:
* разбей данные на три выборки;
* укажи метрику для подбора гиперпараметров, в этом проекте основная метрика — F1-мера;
* поправь расчет AUC-ROC и добавь его на каждом этапе;
* не используй upsample выборки вместе с кросс-валидацией;
* проверь еще хотя бы один способ борьбы с дисбалансом;

Остальные комментарии можно найти в проекте. Готов ответить на любые вопросы :)

<b>Жду доработанный проект на повторное ревью! Удачи!</b>
        
</div>